In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
# Diccionario con ventas simuladas de 6 productos, IVA 21% y 3 departamentos
np.random.seed(42)

productos = ['Producto A', 'Producto B', 'Producto C', 'Producto D', 'Producto E', 'Producto F']
departamentos = ['Electrónica', 'Ropa', 'Hogar']
IVA = 0.21

ventas = {}
for i, producto in enumerate(productos):
    dept = departamentos[i % len(departamentos)]
    precio_unitario = round(np.random.uniform(10, 500), 2)  # precio sin IVA
    cantidad_vendida = int(np.random.randint(1, 200))
    total_sin_iva = round(precio_unitario * cantidad_vendida, 2)
    iva_monto = round(total_sin_iva * IVA, 2)
    total_con_iva = round(total_sin_iva + iva_monto, 2)

    ventas[producto] = {
        'departamento': dept,
        'precio_unitario_sin_iva': precio_unitario,
        'cantidad_vendida': cantidad_vendida,
        'total_sin_iva': total_sin_iva,
        'iva_21%': iva_monto,
        'total_con_iva': total_con_iva
    }

# Mostrar el diccionario generado
ventas

{'Producto A': {'departamento': 'Electrónica',
  'precio_unitario_sin_iva': 193.52,
  'cantidad_vendida': 93,
  'total_sin_iva': 17997.36,
  'iva_21%': 3779.45,
  'total_con_iva': 21776.81},
 'Producto B': {'departamento': 'Ropa',
  'precio_unitario_sin_iva': 99.88,
  'cantidad_vendida': 72,
  'total_sin_iva': 7191.36,
  'iva_21%': 1510.19,
  'total_con_iva': 8701.55},
 'Producto C': {'departamento': 'Hogar',
  'precio_unitario_sin_iva': 303.34,
  'cantidad_vendida': 103,
  'total_sin_iva': 31244.02,
  'iva_21%': 6561.24,
  'total_con_iva': 37805.26},
 'Producto D': {'departamento': 'Electrónica',
  'precio_unitario_sin_iva': 228.46,
  'cantidad_vendida': 75,
  'total_sin_iva': 17134.5,
  'iva_21%': 3598.24,
  'total_con_iva': 20732.74},
 'Producto E': {'departamento': 'Ropa',
  'precio_unitario_sin_iva': 235.03,
  'cantidad_vendida': 117,
  'total_sin_iva': 27498.51,
  'iva_21%': 5774.69,
  'total_con_iva': 33273.2},
 'Producto F': {'departamento': 'Hogar',
  'precio_unitario_sin_iva'

In [3]:
# Convertir el diccionario 'ventas' a un DataFrame
df_ventas = pd.DataFrame.from_dict(ventas, orient='index').reset_index().rename(columns={'index': 'producto'})
df_ventas

,producto,departamento,precio_unitario_sin_iva,cantidad_vendida,total_sin_iva,iva_21%,total_con_iva
0,Producto A,Electrónica,193.52,93,17997.36,3779.45,21776.81
1,Producto B,Ropa,99.88,72,7191.36,1510.19,8701.55
2,Producto C,Hogar,303.34,103,31244.02,6561.24,37805.26
3,Producto D,Electrónica,228.46,75,17134.50,3598.24,20732.74
4,Producto E,Ropa,235.03,117,27498.51,5774.69,33273.20
5,Producto F,Hogar,304.55,152,46291.60,9721.24,56012.84


In [ ]:
# Función para calcular totales de venta desde el dict `ventas` o el DataFrame `df_ventas`
import pandas as pd

def calcular_total_venta(ventas: dict = None,
                          df: pd.DataFrame = None,
                          producto: str = None,
                          incluir_iva: bool = True):
    """
    Calcula totales de venta.
    - Pase `ventas` (dict) o `df` (DataFrame con columna `producto`).
    - Si `producto` se indica, devuelve el total (float) para ese producto.
    - Si no, devuelve totales por producto (dict o pd.Series).
    - `incluir_iva=True` usa la clave/columna `total_con_iva`, sino `total_sin_iva`.
    """
    col = 'total_con_iva' if incluir_iva else 'total_sin_iva'

    # Caso dict
    if ventas is not None:
        if producto:
            item = ventas.get(producto)
            if item is None:
                raise KeyError(f"Producto no encontrado: {producto}")
            return float(item.get(col, item.get(col.replace('total_',''), 0)))
        return {p: float(v.get(col, v.get(col.replace('total_',''), 0))) for p, v in ventas.items()}

    # Caso DataFrame
    if df is not None:
        df2 = df.copy()
        if 'producto' not in df2.columns:
            # intentar tratar el índice como producto
            try:
                df2 = df2.reset_index().rename(columns={'index': 'producto'})
            except Exception:
                pass

        # buscar columna con nombre esperado o alternativa
        if col not in df2.columns:
            alt = col.replace('total_','')
            if alt in df2.columns:
                col = alt
            else:
                raise KeyError(f"No se encontró columna '{col}' ni alternativa en el DataFrame.")

        df2[col] = pd.to_numeric(df2[col], errors='coerce').fillna(0)

        if producto:
            sel = df2[df2['producto'] == producto]
            if sel.empty:
                raise KeyError(f"Producto no encontrado en DataFrame: {producto}")
            return float(sel[col].sum())
        return df2.groupby('producto', dropna=False)[col].sum()

    raise ValueError("Se debe proporcionar `ventas` (dict) o `df` (DataFrame).")


# --- Ejemplos de uso ---
# Asumiendo que existen las variables `ventas` (dict) y `df_ventas` (DataFrame) definidas arriba

# 1) Totales desde el dict
totales_dict = calcular_total_venta(ventas=ventas, incluir_iva=True)
print('Totales (dict):')
print(totales_dict)

# 2) Total de un producto concreto desde el dict
print('\nTotal Producto A (dict):', calcular_total_venta(ventas=ventas, producto='Producto A', incluir_iva=True))

# 3) Totales desde el DataFrame
totales_df = calcular_total_venta(df=df_ventas, incluir_iva=True)
print('\nTotales (DataFrame):')
print(totales_df)

# 4) Total de un producto concreto desde el DataFrame
print('\nTotal Producto B (df):', calcular_total_venta(df=df_ventas, producto='Producto B', incluir_iva=True))
